This notebook uses value iteration to solve the MDP depicted in Figure A1 and produces the results of Table A1.

In [6]:
import numpy as np

In [2]:
class demo_env_min():
    def __init__(self, p_neg, start_val=0, rng=np.random.default_rng(0)) -> None:


        self.p_neg = p_neg
        # states: (state_idx, state_min)
        self.states = [(0,0), 
                       (1,1), (1, -1),
                       (2,0), (2, 1), (2, -2), (2, -1)]
        self.n_states = len(self.states)
        self.start_val = start_val
        self.rng = rng
        # transition_probs: [state_idx][action] = {next_state_idx: prob}
        self.transition_probs = [[{1:0.5, 2:0.5,}], 
                                [{3:1}, {4:1-self.p_neg, 5:self.p_neg,}], [{6:1}, {6:1-self.p_neg, 5:self.p_neg,}], 
                                [],[],[],[]]

        self.reset()

    def get_state_value_idx(self, state):
        idx_mapping= {(0,0): 0, 
                       (1,1): 1, (1, -1): 2,
                       (2,0): 3, (2, 1): 4, (2, -2): 5, (2, -1): 6}
        return idx_mapping[state]

    def get_state_value(self, state):
        return self.state_values[self.get_state_value_idx(state)]

    def reset(self):
        self.state_values = self.start_val * np.zeros(self.n_states)

    def get_state_action_value(self, state, action):
        state_min = state[1]
        action_dict = self.transition_probs[self.states.index(state)][action]

        state_action_val = 0
        for next_state_idx, prob in action_dict.items():
            next_state = self.states[next_state_idx]
            reward = next_state[1] - state_min
            state_action_val += (reward + self.get_state_value(next_state)) * prob
        return state_action_val
    
    def train(self, n_steps):
        
        for _ in range(n_steps):
            for idx, state in enumerate(self.states):
                state_min = state[1]
                action_values = []
                for action_dict in self.transition_probs[idx]:
                    update_val = 0
                    for next_state_idx, prob in action_dict.items():
                        next_state_idx, next_state_min = self.states[next_state_idx]
                        reward = next_state_min - state_min
                        next_state = (next_state_idx, next_state_min)
                        update_val += (reward + self.get_state_value(next_state)) * prob
                    action_values.append(update_val)
                if len(action_values) > 0:
                    self.state_values[self.get_state_value_idx(state)] = np.max(action_values)
                
        return self.state_values

In [3]:
env_min = demo_env_min(p_neg=0.1)
env_min.train(100)

array([-0.15, -0.3 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [4]:
Q_values = {}
Q_values[((0,0), 0)] = env_min.get_state_action_value((0,0), 0)
Q_values[((1,1), 0)] = env_min.get_state_action_value((1,1), 0)
Q_values[((1,1), 1)] = env_min.get_state_action_value((1,1), 1)
Q_values[((1,-1), 0)] = env_min.get_state_action_value((1,-1), 0)
Q_values[((1,-1), 1)] = env_min.get_state_action_value((1,-1), 1)


In [5]:
Q_values

{((0, 0), 0): -0.15000000000000002,
 ((1, 1), 0): -1.0,
 ((1, 1), 1): -0.30000000000000004,
 ((1, -1), 0): 0.0,
 ((1, -1), 1): -0.1}